<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO 5: PROBLEMAS DERIVADOS DE LWE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>



Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuración-previa)
3. [Problema del vector mas cercano](#3.-CVP)
4. [Problema del vector mas corto](#4.-SVP)
5. [Aprendizaje con errores](#5.-LWE)

---
# 1. Introducción






# 2. Configuración previa

In [2]:
#MODULOS A IMPORTAR
import numpy as np
import lattpy as lp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from itertools import product
import time  # Importar time para medir el tiempo
import random

In [3]:
#FUNCIONES UTILIZADAS

def crear_atomos(latt, num_atoms, dimensiones=7):
    """
    Crea átomos en la red con posiciones aleatorias y únicas.
    
    Parámetros:
    latt: objeto de la red donde se agregarán los átomos.
    num_atoms: número de átomos a crear.
    dimensiones: número de dimensiones del espacio (por defecto 7).

    Retorna:
    None
    """
    # Crear un conjunto para asegurarnos de que las posiciones sean únicas
    posiciones_generadas = set()

    while len(posiciones_generadas) < num_atoms:
        # Generar coordenadas aleatorias en [0, 1] para todas las dimensiones
        coords = tuple(np.random.rand(dimensiones))  # np.random.rand genera valores en [0, 1)
        posiciones_generadas.add(coords)  # Añadir a un conjunto asegura unicidad

    # Agregar átomos únicos a la red
    for i, coords in enumerate(posiciones_generadas):
        nombre = f"Atom_{i + 1}"  # Nombra los átomos de forma automática
        latt.add_atom(list(coords), nombre)

# Función para generar los puntos del retículo
def generar_puntos_reticulo(base, limite):
    """
    Genera todos los puntos del retículo dados los vectores base y un límite de los enteros.

    base: matriz de base (n x n) que genera el retículo.
    limite: rango de los valores enteros para las combinaciones.

    Retorna un array con todos los puntos del retículo.
    """
    # Generamos todas las combinaciones posibles de los enteros para k1, k2, ..., kn
    posibles_k = list(product(range(-limite, limite + 1), repeat=base.shape[0]))

    # Generamos los puntos del retículo como combinaciones lineales de los vectores base
    puntos_reticulo = []
    for k in posibles_k:
        punto = sum(k[i] * base[i] for i in range(len(k)))
        puntos_reticulo.append(punto)
    
    return np.array(puntos_reticulo)


def generar_puntos_reticulo_LWE(dim, q):
        """
        Genera puntos del retículo en el espacio de búsqueda.

        Parámetros:
        - dim: dimensión del espacio.
        - q: módulo.

        Retorna:
        - lattice_points: lista de puntos del retículo.
        """
        return list(itertools.product(range(q), repeat=dim))

#Funcion para calcular el vector mas cercano dado un punto
def closestVector(base, limite, puntos_reticulo, target_point):
    """
    Encuentra el vector más cercano desde un punto objetivo (target_point) a todos los puntos del retículo.
    
    base: matriz de base del retículo.
    limite: rango de los índices.
    puntos_reticulo: arreglo con los puntos del retículo generados.
    target_point: punto objetivo desde el cual buscar el vector más cercano.
    
    Retorna el vector más cercano y su norma.
    """
    
    # Inicializar variables para almacenar el mejor vector y su norma más baja
    mejor_vector = None
    menor_norma = float('inf')

    # Iterar sobre todos los puntos del retículo
    for punto in puntos_reticulo:
        if not np.array_equal(punto, target_point):  # Evitar el propio punto objetivo
            vector = punto - target_point  # Vector entre el punto objetivo y el punto del retículo
            norma = np.linalg.norm(vector)  # Calcular la norma del vector

            # Si encontramos un vector con norma menor, lo actualizamos
            if norma < menor_norma:
                mejor_vector = vector
                menor_norma = norma
    
    return mejor_vector, menor_norma


# Función para calcular el vector más corto entre todos los puntos del retículo
def shortestVector(base, limite, puntos_reticulo):
    """
    Encuentra el vector más corto entre todos los puntos del retículo generado por los vectores base,
    excluyendo el origen. Considera todas las combinaciones posibles de pares de puntos.

    base: matriz de base del retículo.
    limite: rango de los índices.

    Retorna el vector más corto y su norma.
    """
    
    # Inicializar variables para almacenar el mejor vector y su norma más baja
    mejor_vector = None
    menor_norma = float('inf')

    # Iterar sobre todos los pares de puntos posibles
    for i, punto_1 in enumerate(puntos_reticulo):
        for j, punto_2 in enumerate(puntos_reticulo):
            if i != j:  # Evitar calcular el vector entre un punto y él mismo
                vector = punto_2 - punto_1  # Vector entre el par de puntos
                norma = np.linalg.norm(vector)  # Calcular la norma del vector
                
                # Si encontramos un vector con norma menor, lo actualizamos
                if norma < menor_norma:
                    mejor_vector = vector
                    menor_norma = norma
    
    return mejor_vector, menor_norma




In [4]:
class Ring:
    def __init__(self, elements , add, mul, zero, one=None):
        """
        Representa un anillo algebraico.
        
        :param elements: Conjunto de elementos del anillo.
        :param add: Función binaria que define la adición.
        :param mul: Función binaria que define la multiplicación.
        :param zero: Elemento neutro aditivo.
        :param one: (Opcional) Elemento neutro multiplicativo.
        """
        self.elements = set(elements)
        self.add = add
        self.mul = mul
        self.zero = zero
        self.one = one

        if zero not in elements:
            raise ValueError("El elemento neutro aditivo debe estar en el conjunto de elementos.")

        if one and one not in elements:
            raise ValueError("El elemento neutro multiplicativo debe estar en el conjunto de elementos.")

    # Propiedades de la suma
    def is_closed_under_addition(self):
        return all(self.add(a, b) in self.elements for a in self.elements for b in self.elements)

    def is_addition_associative(self):
        return all(self.add(self.add(a, b), c) == self.add(a, self.add(b, c))
                   for a in self.elements for b in self.elements for c in self.elements)

    def is_addition_commutative(self):
        return all(self.add(a, b) == self.add(b, a) for a in self.elements for b in self.elements)

    def has_additive_identity(self):
        return all(self.add(a, self.zero) == a for a in self.elements)

    def has_additive_inverses(self):
        return all(any(self.add(a, b) == self.zero for b in self.elements) for a in self.elements)

    # Propiedades del producto 
    def is_closed_under_multiplication(self):
        return all(self.mul(a, b) in self.elements for a in self.elements for b in self.elements)

    def is_multiplication_associative(self):
        return all(self.mul(self.mul(a, b), c) == self.mul(a, self.mul(b, c))
                   for a in self.elements for b in self.elements for c in self.elements)

    def is_distributive(self):
        return all(
            self.mul(a, self.add(b, c)) == self.add(self.mul(a, b), self.mul(a, c)) and
            self.mul(self.add(a, b), c) == self.add(self.mul(a, c), self.mul(b, c))
            for a in self.elements for b in self.elements for c in self.elements
        )

    # FUNCIONES PARA ANILLOS UNITARIOS
    def has_multiplication_identity(self):
        return all(self.mul(a, self.one) == a for a in self.elements)

    def is_unital_ring(self):
        return Ring.has_multiplication_identity(self)
        
    # FUNCIONES PARA ANILLOS CONMUTATIVOS
    def is_multiplication_conmutative(self):
        return all(self.mul(a, b) == self.mul(b, a) for a in self.elements for b in self.elements)

    def is_conmutative_ring(self):
        return Ring.is_multiplication_conmutative(self)

    # Método para verificar todas las propiedades del anillo
    def test_ring_properties(self):
        print("Propiedades de la suma")
        print("  ¿Cerradura bajo la adición?:", self.is_closed_under_addition())
        print("  ¿La suma es asociativa?:", self.is_addition_associative())
        print("  ¿La suma es conmutativa?:", self.is_addition_commutative())
        print("  ¿Tiene elemento neutro aditivo?:", self.has_additive_identity())
        print("  ¿Todos los elementos tienen inverso aditivo?:", self.has_additive_inverses())
        
        print("\nPropiedades del producto")
        print("  ¿Cerradura bajo la multiplicación?:", self.is_closed_under_multiplication())
        print("  ¿El producto es asociativo?:", self.is_multiplication_associative())
        print("  ¿El producto es distributivo respecto a la suma?:", self.is_distributive())


In [5]:
def generate_LWE_samples(secret, num_samples, dim, max_value, noise_stddev):
    """
    Genera muestras LWE.

    Parámetros:
    - secret: vector secreto.
    - num_samples: número de muestras.
    - dim: dimensión del secreto.
    - max_value: rango máximo de valores para los vectores 'a'.
    - noise_stddev: desviación estándar del ruido.

    Retorna:
    - samples: lista de tuplas (a, b).
    """
    samples = []

    for _ in range(num_samples):
        # Generar vector 'a' aleatorio en el rango [-max_value, max_value]
        a = [random.randint(-max_value, max_value) for _ in range(dim)]

        # Calcular b = a . s + e
        b = sum(a[i] * secret[i] for i in range(dim))

        # Añadir ruido aleatorio en el rango [-noise_stddev, noise_stddev]
        error = random.randint(-noise_stddev, noise_stddev)
        b += error

        # Agregar la muestra a la lista
        samples.append((a, b))

    return samples


def solve_LWE(samples, lattice_points, noise_stddev):
    """
    Resuelve LWE buscando entre los puntos del retículo.

    Parámetros:
    - samples: lista de tuplas (a, b).
    - lattice_points: puntos del retículo.
    - noise_stddev: tolerancia de ruido.

    Retorna:
    - solution: vector secreto aproximado o None si no se encuentra solución.
    """
    dim = len(lattice_points[0])

    while True:
        try:
            for candidate in lattice_points:
                valid = True

                for a, b in samples:
                    computed = sum(a[j] * candidate[j] for j in range(dim))

                    if abs(computed - b) > noise_stddev:
                        valid = False
                        break

                if valid:
                    return candidate  # Solución encontrada
        except RecursionError:
            print("Error: RecursionError. Retomando ejecución.")

---

# 3. RLWE

a

In [6]:
import numpy as np
import lattpy as lp

# Parámetros
n = 8  # Grado del polinomio
q = 256  # Modulo
t = 2  # Valor máximo para los coeficientes del mensaje

# Generación de polinomios aleatorios
def generate_polynomial(n, q):
    return np.random.randint(0, q, n)

# Generación de un polinomio de error pequeño
def generate_error_polynomial(n, q):
    return np.random.randint(-1, 2, n)  # Errores pequeños: -1, 0 o 1

# Generación del polinomio público
def generate_public_polynomial(n, q):
    return np.random.randint(0, q, n)

# Función de encriptación
def encrypt(public_polynomial, secret_polynomial, error_polynomial, q):
    b = np.polyadd(np.polymul(public_polynomial, secret_polynomial), error_polynomial)
    return b % q

# Función de desencriptación
def decrypt(ciphertext, secret_polynomial, q):
    result = np.polydiv(ciphertext, secret_polynomial)
    return result[0] % q

# Generación de polinomios
s = generate_polynomial(n, q)  # Polinomio secreto
e = generate_error_polynomial(n, q)  # Polinomio de error
a = generate_public_polynomial(n, q)  # Polinomio público

# Encriptación
b = encrypt(a, s, e, q)

# Desencriptación
recovered_s = decrypt(b, s, q)

print("Polinomio secreto original:", s)
print("Polinomio secreto recuperado:", recovered_s)



Polinomio secreto original: [149 156   8  98  89 230  59   8]
Polinomio secreto recuperado: [  1.27516779 254.69848205   2.47540576 253.23008324   4.21719862
 249.47136303   9.00660898 243.8434907 ]


In [9]:
import numpy as np
import random
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt

# Función para generar un polinomio aleatorio mod q
def generate_random_polynomial(degree, q):
    return Polynomial([random.randint(0, q - 1) for _ in range(degree + 1)])

# Función para reducir coeficientes módulo q
def reduce_mod_q(poly, q):
    return Polynomial([coef % q for coef in poly.coef])

# Función para generar ruido pequeño
def generate_noise(degree, noise_stddev):
    return Polynomial([int(random.gauss(0, noise_stddev)) for _ in range(degree + 1)])

# Generar muestras RLWE
def generate_RLWE_samples(secret, num_samples, degree, q, noise_stddev):
    samples = []
    
    for _ in range(num_samples):
        # Generar polinomio aleatorio "a"
        a = generate_random_polynomial(degree, q)

        # Generar ruido pequeño "e"
        e = generate_noise(degree, noise_stddev)

        # Calcular b = a * secret + e (mod q)
        b = reduce_mod_q(a * secret + e, q)

        # Asegurar que los polinomios b y a tengan el mismo grado
        b = Polynomial(np.pad(b.coef, (0, max(0, len(a.coef) - len(b.coef))), constant_values=0))

        samples.append((a, b))

    return samples

# Resolver RLWE buscando entre posibles secretos
# En esta implementación, asumimos un espacio reducido para posibles secretos
# Esto es solo para propósitos educativos; RLWE real requiere búsqueda más eficiente.
def solve_RLWE(samples, candidates, q):
    for candidate in candidates:
        valid = True

        for a, b in samples:
            # Calcular a * candidato mod q y asegurar la misma longitud que b
            candidate_poly = reduce_mod_q(a * candidate, q)
            candidate_poly = Polynomial(np.pad(candidate_poly.coef, (0, max(0, len(b.coef) - len(candidate_poly.coef))), constant_values=0))

            # Comprobar si b = a * candidate (mod q)
            if not np.allclose(candidate_poly.coef[:len(b.coef)], b.coef, atol=1e-3):
                valid = False
                break

        if valid:
            return candidate

    return None

# Parámetros del problema RLWE
degree = 8          # Grado del polinomio
num_samples = 10    # Número de muestras
q = 23              # Módulo
noise_stddev = 1.5  # Desviación estándar del ruido

# Generar secreto aleatorio
secret = generate_random_polynomial(degree, q)
print(f"Secreto original: {secret}")

# Generar muestras RLWE
samples = generate_RLWE_samples(secret, num_samples, degree, q, noise_stddev)

# Generar candidatos para la búsqueda (reducción del espacio de búsqueda para fines educativos)
candidates = [generate_random_polynomial(degree, q) for _ in range(100)]

# Resolver RLWE
recovered_secret = solve_RLWE(samples, candidates, q)

if recovered_secret:
    print(f"Secreto recuperado: {recovered_secret}")
else:
    print("No se pudo recuperar el secreto.")



Secreto original: 13.0 + 12.0·x + 9.0·x² + 3.0·x³ + 17.0·x⁴ + 18.0·x⁵ + 7.0·x⁶ + 8.0·x⁷ +
8.0·x⁸
No se pudo recuperar el secreto.


---
# 4. MLWE


a


In [11]:
import numpy as np
import random

def generate_secret_and_error(n, q):
    """
    Genera un vector secreto y un vector de errores.
    """
    secret = np.random.randint(0, q, n)  # Genera el vector secreto (de dimensión n)
    error = np.random.randint(-q//2, q//2, n)  # Genera un vector de errores pequeños
    return secret, error

def generate_A_and_b(secret, n, m, q):
    """
    Genera una instancia del problema MLWE.
    - A: Matriz de dimensión m x n
    - b: Vector de dimensión m
    """
    # A es una matriz aleatoria de tamaño m x n
    A = np.random.randint(0, q, (m, n))
    
    # Genera el vector b = A * secret + error (modulo q)
    error = np.random.randint(-q//2, q//2, m)  # Error de dimensión m
    b = (np.dot(A, secret) + error) % q  # b es la combinación lineal más el error
    return A, b

def solve_lwe(A, b, q):
    """
    Función muy simplificada para resolver el problema LWE (o MLWE).
    Esto no es una solución real, solo una ilustración.
    """
    # Una técnica simplificada para encontrar un vector cercano a la solución
    # Vamos a intentar reconstruir el vector secreto con un enfoque naivo (por ejemplo, buscando el valor más cercano)
    solution = np.zeros(A.shape[1], dtype=int)
    for i in range(A.shape[1]):
        column = A[:, i]
        value = np.dot(column, b) % q
        solution[i] = value % q
    return solution

def main():
    # Parámetros
    n = 5  # Dimensión del vector secreto
    m = 7  # Número de ecuaciones
    q = 101  # Modulo (un número primo)
    
    # Generación de la instancia MLWE
    secret, error = generate_secret_and_error(n, q)
    A, b = generate_A_and_b(secret, n, m, q)
    
    print("Secreto original:", secret)
    print("Matriz A:\n", A)
    print("Vector b:", b)
    
    # Resolver el problema MLWE (en este caso, simplemente intentar encontrar una aproximación del secreto)
    solution = solve_lwe(A, b, q)
    
    print("Solución aproximada:", solution)

if __name__ == "__main__":
    main()




Secreto original: [37 49 56 38 39]
Matriz A:
 [[ 84  45  32  74  12]
 [ 83  98   1  51  78]
 [ 39  51 100   9  50]
 [ 86  37  19  86   2]
 [ 63   0  10  14  22]
 [ 98   2  85  26  40]
 [ 32   6  99  23  86]]
Vector b: [50 99 88 52 60 82 47]
Solución aproximada: [ 8 24 76 88 56]
